## Depth integrated tracer on shelf per km along shelf
This notebook has plots for showing the canyon effect on total tracer mass on shelf and volume of HCW along the coast for all runs. **Canyon effect** is defined as the residual from subtracting a flat shelf run from a canyon run with the exact same configuration (other than not having a canyon). 

In [2]:
#import gsw as sw # Gibbs seawater package

from math import *

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

%matplotlib inline

#from MITgcmutils import rdmds

from netCDF4 import Dataset

import numpy as np

import os 

import pandas as pd

import pylab as pl

import scipy.io

import scipy as spy

import seaborn as sns

import sys

In [3]:
lib_path = os.path.abspath('../../Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
#lib_path = os.path.abspath('../BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts

sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 
import MetricsPythonTools as mpt


In [4]:
sns.set()
sns.set_style('white')
sns.set_context('talk')

In [5]:
# plotting functions
def plotCanyonPos(ax,UpX,DnX):
    ax.axvline(UpX,color='0.70',linestyle='--')
    ax.axvline(DnX,color='0.70',linestyle='--')


In [6]:
#Exp
CGrid = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03/gridGlob.nc' 
CGridOut = Dataset(CGrid)
for dimobj in CGridOut.dimensions.values():
    print dimobj

CGridNoC = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run07/gridGlob.nc' 
CGridNoCOut = Dataset(CGridNoC)
#------------------------------------------------------------------------------------
Ptracers = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03/ptracersGlob.nc' 
PtracersOut = Dataset(Ptracers)

PtracersNoC = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run07/ptracersGlob.nc' #No canyon
PtracersOutNoC = Dataset(PtracersNoC)
#------------------------------------ Kv ----------------------------------------------
Ptracers11 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run11/ptracersGlob.nc' 
Ptracers12 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run12/ptracersGlob.nc' 

Ptracers24 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run24/ptracersGlob.nc' #No canyon
Ptracers25 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run25/ptracersGlob.nc' #No canyon
#----------------------------------- N -----------------------------------------------
Ptracers14 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run14/ptracersGlob.nc' 
Ptracers15 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run15/ptracersGlob.nc' 
Ptracers16 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run16/ptracersGlob.nc' 
Ptracers17 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run17/ptracersGlob.nc' 

Ptracers26 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run26/ptracersGlob.nc' #No canyon
Ptracers27 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run27/ptracersGlob.nc' #No canyon
Ptracers28 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run28/ptracersGlob.nc' #No canyon
Ptracers30 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run30/ptracersGlob.nc' #No canyon

#---------------------------------- f --------------------------------------------------
Ptracers18 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run18/ptracersGlob.nc' 
Ptracers19 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run19/ptracersGlob.nc' 

Ptracers22 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run22/ptracersGlob.nc' #No canyon
Ptracers23 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run23/ptracersGlob.nc' #No canyon
#---------------------------------- Kv 3D ----------------------------------------------
Ptracers043D = '/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run04/ptracersGlob.nc' 
Ptracers053D = '/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run05/ptracersGlob.nc' 
Ptracers063D = '/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run06/ptracersGlob.nc' 
Ptracers073D = '/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run07/ptracersGlob.nc' 


State = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03/stateGlob.nc' 
StateNoC = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run07/stateGlob.nc' 




<type 'netCDF4.Dimension'>: name = 'Z', size = 90

<type 'netCDF4.Dimension'>: name = 'Zp1', size = 91

<type 'netCDF4.Dimension'>: name = 'Zu', size = 90

<type 'netCDF4.Dimension'>: name = 'Zl', size = 90

<type 'netCDF4.Dimension'>: name = 'X', size = 360

<type 'netCDF4.Dimension'>: name = 'Y', size = 360

<type 'netCDF4.Dimension'>: name = 'Xp1', size = 361

<type 'netCDF4.Dimension'>: name = 'Yp1', size = 361



In [7]:
# General input

nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 
numTr = 22 # number of tracers in total (CNT =22, 3D = 4, total = 19)

rc = CGridNoCOut.variables['RC']
dxf = CGridNoCOut.variables['dxF']
xc = rout.getField(CGridNoC, 'XC') # x coords tracer cells
yc = rout.getField(CGridNoC, 'YC') # y coords tracer cells

rA = rout.getField(CGridNoC, 'rA') 

drF = CGridNoCOut.variables['drF'] # vertical distance between faces
drC = CGridNoCOut.variables['drC'] # vertical distance between centers

hFacC = rout.getField(CGridNoC, 'HFacC')
mask_NoC = rout.getMask(CGridNoC, 'HFacC')

times = np.arange(0,nt,1)

#print(drC[:])
#print(np.shape(drC))

In [8]:
def HCW_Tracer_AlongShelf(Tr,MaskC,nzlim,rA,hFacC,drF,yin,zfin,xi,yi):
  '''
  INPUT----------------------------------------------------------------------------------------------------------------
    Tr    : Array with concentration values for a tracer. Until this function is more general, this should be size 19x90x360x360
    MaskC : Land mask for tracer
    nzlim : The nz index under which to look for water properties
    rA    : Area of cell faces at C points (360x360)
    fFacC : Fraction of open cell (90x360x360)
    drF   : Distance between cell faces (90)
    yin   : across-shore index of shelf break
    zfin  : shelf break index + 1 
    xi    : initial profile x index
    yi    : initial profile y index
    
    OUTPUT----------------------------------------------------------------------------------------------------------------
    VolWaterHighConc =  Array with the volume of water over the shelf [t,360] at every time output.
    Total_Tracer =  Array with the mass of tracer (m^3*[C]*l/m^3) at each x-position over the shelf [t,360] at 
                    every time output. 
                                                
  -----------------------------------------------------------------------------------------------------------------------
  '''
  maskExp = mpt.maskExpand(MaskC,Tr)

  TrMask=np.ma.array(Tr,mask=maskExp)   
    
  trlim = TrMask[0,nzlim,yi,xi]
    
  print('tracer limit concentration is: ',trlim)
    
    
  # mask cells with tracer concentration < trlim on shelf
  HighConc_Masked = np.ma.masked_less(TrMask[:,:zfin,yin:,:], trlim) 
  HighConc_Mask = HighConc_Masked.mask
    
  #Get volume of water of cells with relatively high concentration
  rA_exp = np.expand_dims(rA[yin:,:],0)
  drF_exp = np.expand_dims(np.expand_dims(drF[:zfin],1),1)
  rA_exp = rA_exp + np.zeros(hFacC[:zfin,yin:,:].shape)
  drF_exp = drF_exp + np.zeros(hFacC[:zfin,yin:,:].shape)
    
  ShelfVolume = hFacC[:zfin,yin:,:]*drF_exp*rA_exp
  ShelfVolume_exp = np.expand_dims(ShelfVolume,0)
  ShelfVolume_exp = ShelfVolume_exp + np.zeros(HighConc_Mask.shape)
    
  HighConc_CellVol = np.ma.masked_array(ShelfVolume_exp,mask = HighConc_Mask) 
  VolWaterHighConc =np.ma.sum(np.ma.sum(HighConc_CellVol,axis = 1),axis=1)
    
   #Get total mass of tracer on shelf
  Total_Tracer = np.ma.sum(np.ma.sum(ShelfVolume_exp*TrMask[:,:zfin,yin:,:]*1000.0,axis = 1),axis=1)
   # 1 m^3 = 1000 l
    
  return (VolWaterHighConc, Total_Tracer)

In [9]:
Tra = rout.getField(Ptracers, 'Tr1')
Trb = rout.getField(PtracersNoC, 'Tr1')

In [ ]:
HCW,TrMass = HCW_Tracer_AlongShelf(Tra,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCWNoC,TrMassNoC = HCW_Tracer_AlongShelf(Trb,mask_NoC,30,rA,hFacC,drF,227,30,180,50)

('tracer limit concentration is: ', 7.3974895)
('tracer limit concentration is: ', 7.3974895)


## Changing Diffusivity

In [ ]:
Tra = rout.getField(Ptracers11, 'Tr2')
Trb = rout.getField(Ptracers12, 'Tr3')
Trc = rout.getField(Ptracers24, 'Tr1')
Trd = rout.getField(Ptracers25, 'Tr1')
Tre = rout.getField(Ptracers17, 'Tr1')
Trf = rout.getField(Ptracers30, 'Tr1')

In [ ]:
HCW11,TrMass11 = HCW_Tracer_AlongShelf(Tra,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW12,TrMass12 = HCW_Tracer_AlongShelf(Trb,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW17,TrMass17 = HCW_Tracer_AlongShelf(Tre,mask_NoC,30,rA,hFacC,drF,227,30,180,50)

HCW24,TrMass24 = HCW_Tracer_AlongShelf(Trc,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW25,TrMass25 = HCW_Tracer_AlongShelf(Trd,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW30,TrMass30 = HCW_Tracer_AlongShelf(Trf,mask_NoC,30,rA,hFacC,drF,227,30,180,50)

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (14,12),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(HCW17[ii,:]-HCW30[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["forest"],label='$\kappa_v=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $\kappa_v=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW11[ii,:]-HCW24[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["leaf"],label='$\kappa_v=10^{-4}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW12[ii,:]-HCW25[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["jade"],label='$\kappa_v=10^{-3}m^2s^{-1}$')
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax1.legend(loc=0)
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Volume HCW per m ($10^3$ $m^3$)')

    
plt.tight_layout()

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (14,12),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(TrMass17[ii,:]-TrMass30[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["forest"],label='$\kappa_v=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $\kappa_v=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass11[ii,:]-TrMass24[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["leaf"],label='$\kappa_v=10^{-4}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass12[ii,:]-TrMass25[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["jade"],label='$\kappa_v=10^{-3}m^2s^{-1}$')
   
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax1.legend(loc=0)
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Tracer mass per m ($10^6$ $[C]m^3$)')

plt.tight_layout()    


In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1, figsize = (10,10),sharex=True)

tt = np.arange(19)

ax1.plot(tt/2.,np.sum(TrMass17-TrMass30,axis=1)/1.E14,sns.xkcd_rgb["forest"],marker = 'o',label='$\kappa_v=10^{-7}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass-TrMassNoC,axis=1)/1.E14,sns.xkcd_rgb["kelly green"],marker = 'o',label='Base $\kappa_v=10^{-5}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass11-TrMass24,axis=1)/1.E14,sns.xkcd_rgb["leaf"],marker = 'o',label='$\kappa_v=10^{-4}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass12-TrMass25,axis=1)/1.E14,sns.xkcd_rgb["jade"],marker = 'o',label='$\kappa_v=10^{-3}m^2s^{-1}$')
   
ax2.plot(tt/2.,np.sum(HCW17-HCW30,axis=1)/1.E10,sns.xkcd_rgb["forest"],marker = 'o',label='$\kappa_v=10^{-7}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW-HCWNoC,axis=1)/1.E10,sns.xkcd_rgb["kelly green"],marker = 'o',label='Base $\kappa_v=10^{-5}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW11-HCW24,axis=1)/1.E10,sns.xkcd_rgb["leaf"],marker = 'o',label='$\kappa_v=10^{-4}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW12-HCW25,axis=1)/1.E10,sns.xkcd_rgb["jade"],marker = 'o',label='$\kappa_v=10^{-3}m^2s^{-1}$')
  
    
ax1.legend(loc=0)
ax2.set_xlabel('Day')
ax1.set_ylabel('Tracer mass per m ($10^{14}$ $[C]m^3$)')
ax2.set_ylabel('HCW volume per m ($10^{10}$ $m^3$)')
plt.tight_layout()

plt.show()



## Changing geography of diapycnal diffusivity 

In [ ]:
Tra = rout.getField(Ptracers043D, 'Tr1')
Trb= rout.getField(Ptracers053D, 'Tr1')
Trc = rout.getField(Ptracers063D, 'Tr1')
Trd = rout.getField(Ptracers073D, 'Tr1')

In [ ]:
HCW043D,TrMass043D = HCW_Tracer_AlongShelf(Tra,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW053D,TrMass053D = HCW_Tracer_AlongShelf(Trb,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW063D,TrMass063D = HCW_Tracer_AlongShelf(Trc,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW073D,TrMass073D = HCW_Tracer_AlongShelf(Trd,mask_NoC,30,rA,hFacC,drF,227,30,180,50)

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (14,12),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(HCW[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $\kappa_v=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW043D[ii,:]-HCW30[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["royal purple"],label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW053D[ii,:]-HCW30[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["muted purple"],label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW063D[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["dark hot pink"],label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW073D[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["muted pink"],label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')
    
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax1.legend(loc=0)
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Volume HCW per m ($10^3$ $m^3$)')

plt.tight_layout()    


In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (14,12),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(TrMass[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $\kappa_v=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass043D[ii,:]-TrMass30[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["royal purple"],label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass053D[ii,:]-TrMass30[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["muted purple"],label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass063D[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["dark hot pink"],label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass073D[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["muted pink"],label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')
    
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax1.legend(loc=0)
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Tracer mass per m ($10^6$ $[C]m^3$)')

    
plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1, figsize = (10,10),sharex=True)

tt = np.arange(19)

ax1.plot(tt/2.,np.sum(TrMass-TrMassNoC,axis=1)/1.E14,sns.xkcd_rgb["kelly green"],marker = 'o',label='Base $\kappa_v=10^{-5}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass043D-TrMass30,axis=1)/1.E14,sns.xkcd_rgb["royal purple"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass053D-TrMass30,axis=1)/1.E14,sns.xkcd_rgb["muted purple"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass063D-TrMassNoC,axis=1)/1.E14,sns.xkcd_rgb["dark hot pink"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass073D-TrMassNoC,axis=1)/1.E14,sns.xkcd_rgb["muted pink"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')
   
ax2.plot(tt/2.,np.sum(HCW-HCWNoC,axis=1)/1.E10,sns.xkcd_rgb["kelly green"],marker = 'o',label='Base $\kappa_v=10^{-5}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW043D-HCW30,axis=1)/1.E10,sns.xkcd_rgb["royal purple"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW053D-HCW30,axis=1)/1.E10,sns.xkcd_rgb["muted purple"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW063D-HCWNoC,axis=1)/1.E10,sns.xkcd_rgb["dark hot pink"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW073D-HCWNoC,axis=1)/1.E10,sns.xkcd_rgb["muted pink"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')
  
    
ax1.legend(loc=0)
ax2.set_xlabel('Day')
ax1.set_ylabel('Tracer mass per m ($10^{14}$ $[C]m^3$)')
ax2.set_ylabel('HCW volume per m ($10^{10}$ $m^3$)')
plt.tight_layout()

plt.show()


## Changing initial stratification

In [ ]:
Tra = rout.getField(Ptracers14, 'Tr1')
Trb = rout.getField(Ptracers15, 'Tr1')
Trc = rout.getField(Ptracers16, 'Tr1')
Trd = rout.getField(Ptracers26, 'Tr1')
Tre = rout.getField(Ptracers27, 'Tr1')
Trf = rout.getField(Ptracers28, 'Tr1')


In [ ]:
HCW14,TrMass14 = HCW_Tracer_AlongShelf(Tra,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW15,TrMass15 = HCW_Tracer_AlongShelf(Trb,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW16,TrMass16 = HCW_Tracer_AlongShelf(Trc,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW26,TrMass26 = HCW_Tracer_AlongShelf(Trd,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW27,TrMass27 = HCW_Tracer_AlongShelf(Tre,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW28,TrMass28 = HCW_Tracer_AlongShelf(Trf,mask_NoC,30,rA,hFacC,drF,227,30,180,50)


In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (14,12),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(HCW16[ii,:]-HCW28[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["bright red"],label='$N=3.0E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW14[ii,:]-HCW26[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["blood orange"],label='$N=3.9E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $N=5.5E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW15[ii,:]-HCW27[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["saffron"],label='$N=6.3E-3s^{-1}$')
    
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax1.legend(loc=0)
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Volume HCW per m ($10^3$ $m^3$)')
plt.tight_layout()


In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (14,12),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(TrMass16[ii,:]-TrMass28[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["bright red"],label='$N=3.0E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass14[ii,:]-TrMass26[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["blood orange"],label='$N=3.9E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $N=5.5E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass15[ii,:]-TrMass27[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["saffron"],label='$N=6.3E-3s^{-1}$')
   
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax1.legend(loc=0)
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Tracer mass per m ($10^6$ $[C]m^3$)')

plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1, figsize = (10,10),sharex=True)

tt = np.arange(19)

ax1.plot(tt/2.,np.sum(TrMass16-TrMass28 ,axis=1)/1.E14,sns.xkcd_rgb["bright red"],label='$N=3.0E-3s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass14-TrMass26 ,axis=1)/1.E14,sns.xkcd_rgb["blood orange"],label='$N=3.9E-3s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass-TrMassNoC ,axis=1)/1.E14,sns.xkcd_rgb["kelly green"],label='Base $N=5.5E-3s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass15-TrMass27 ,axis=1)/1.E14,sns.xkcd_rgb["saffron"],label='$N=6.3E-3s^{-1}$')
   
ax2.plot(tt/2.,np.sum(HCW16-HCW28 ,axis=1)/1.E10,sns.xkcd_rgb["bright red"],label='$N=3.0E-3s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW14-HCW26 ,axis=1)/1.E10,sns.xkcd_rgb["blood orange"],label='$N=3.9E-3s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW-HCWNoC ,axis=1)/1.E10,sns.xkcd_rgb["kelly green"],label='Base $N=5.5E-3s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW15-HCW27 ,axis=1)/1.E10,sns.xkcd_rgb["saffron"],label='$N=6.3E-3s^{-1}$')
     
    
ax1.legend(loc=0)
ax2.set_xlabel('Day')
ax1.set_ylabel('Tracer mass per m ($10^{14}$ $[C]m^3$)')
ax2.set_ylabel('HCW volume per m ($10^{10}$ $m^3$)')
plt.tight_layout()
plt.show()



## Changing latitude (f)

In [ ]:
Tra = rout.getField(Ptracers18, 'Tr1')
Trb = rout.getField(Ptracers19, 'Tr1')
Trc = rout.getField(Ptracers22, 'Tr1')
Trd = rout.getField(Ptracers23, 'Tr1')

In [ ]:
HCW18,TrMass18 = HCW_Tracer_AlongShelf(Tra,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW19,TrMass19 = HCW_Tracer_AlongShelf(Trb,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW22,TrMass22 = HCW_Tracer_AlongShelf(Trc,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW23,TrMass23 = HCW_Tracer_AlongShelf(Trd,mask_NoC,30,rA,hFacC,drF,227,30,180,50)


In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (13,11),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(HCW[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $f=9.66E-5s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW18[ii,:]-HCW22[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["sapphire"],label='$f=7.68E-5s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW19[ii,:]-HCW23[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["cerulean"],label='$f=4.84E-5s^{-1}$')
    
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax1.legend(loc=0)
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Volume HCW per m ($10^3$ $m^3$)')
plt.tight_layout()

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (13,11),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(TrMass[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $f=9.66E-5s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass18[ii,:]-TrMass22[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["sapphire"],label='$f=7.68E-5s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass19[ii,:]-TrMass23[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["cerulean"],label='$f=4.84E-5s^{-1}$')
    
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax1.legend(loc=0)
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Tracer mass per m ($10^6$ $[C]m^3$)')

plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1, figsize = (10,10),sharex=True)

tt = np.arange(19)


ax2.plot(tt/2.,np.sum(HCW-HCWNoC,axis=1)/1.E10,sns.xkcd_rgb["kelly green"],label='Base $f=9.66E-5s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW18-HCW22,axis=1)/1.E10,sns.xkcd_rgb["sapphire"],label='$f=7.68E-5s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW19-HCW23,axis=1)/1.E10,sns.xkcd_rgb["cerulean"],label='f=4.84E-5s^{-1}$')
 
ax1.plot(tt/2.,np.sum(TrMass-TrMassNoC,axis=1)/1.E14,sns.xkcd_rgb["kelly green"],label='Base $f=9.66E-5s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass18-TrMass22,axis=1)/1.E14,sns.xkcd_rgb["sapphire"],label='$f=7.68E-5s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass19-TrMass23,axis=1)/1.E14,sns.xkcd_rgb["cerulean"],label='$f=4.84E-5s^{-1}$')
        
    
ax1.legend(loc=0)
ax2.set_xlabel('Day')
ax1.set_ylabel('Tracer mass per m ($10^{14}$ $[C]m^3$)')
ax2.set_ylabel('HCW volume per m ($10^{10}$ $m^3$)')
plt.tight_layout()
plt.show()

## All variations together

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (13,11),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(HCW[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $f=9.66E-5s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW17[ii,:]-HCW30[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["forest"],label='$\kappa_v=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW11[ii,:]-HCW24[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["leaf"],label='$\kappa_v=10^{-4}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW12[ii,:]-HCW25[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["jade"],label='$\kappa_v=10^{-3}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW043D[ii,:]-HCW30[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["royal purple"],label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW053D[ii,:]-HCW30[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["muted purple"],label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW063D[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["dark hot pink"],label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW073D[ii,:]-HCWNoC[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["muted pink"],label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW16[ii,:]-HCW28[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["bright red"],label='$N=3.0E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW14[ii,:]-HCW26[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["blood orange"],label='$N=3.9E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW15[ii,:]-HCW27[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["saffron"],label='$N=6.3E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW18[ii,:]-HCW22[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["sapphire"],label='$f=7.68E-5s^{-1}$')
    ax.plot(xc[1,:]/1000,(HCW19[ii,:]-HCW23[ii,:])/(1000*dxf[0,:]),sns.xkcd_rgb["cerulean"],label='$f=4.84E-5s^{-1}$')
    
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax3.legend(loc ='upper right', bbox_to_anchor=(2,1))
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Volume HCW per m ($10^3$ $m^3$)')
plt.tight_layout()



In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3, figsize = (13,11),sharex=True, sharey=True)

axlist = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]

times = [2,4,6,8,10,12,14,16,18]
for ii,ax in zip(times,axlist):
    ax.axhline(0,color='0.70',linestyle='-')
    ax.plot(xc[1,:]/1000,(TrMass[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["kelly green"],label='Base $f=9.66E-5s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass17[ii,:]-TrMass30[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["forest"],label='$\kappa_v=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass11[ii,:]-TrMass24[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["leaf"],label='$\kappa_v=10^{-4}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass12[ii,:]-TrMass25[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["jade"],label='$\kappa_v=10^{-3}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass043D[ii,:]-TrMass30[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["royal purple"],label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass053D[ii,:]-TrMass30[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["muted purple"],label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass063D[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["dark hot pink"],label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass073D[ii,:]-TrMassNoC[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["muted pink"],label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass16[ii,:]-TrMass28[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["bright red"],label='$N=3.0E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass14[ii,:]-TrMass26[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["blood orange"],label='$N=3.9E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass15[ii,:]-TrMass27[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["saffron"],label='$N=6.3E-3s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass18[ii,:]-TrMass22[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["sapphire"],label='$f=7.68E-5s^{-1}$')
    ax.plot(xc[1,:]/1000,(TrMass19[ii,:]-TrMass23[ii,:])/(1000000*dxf[0,:]),sns.xkcd_rgb["cerulean"],label='$f=4.84E-5s^{-1}$')
    
    
    ax.set_title('day %d' %(ii/2))
    plotCanyonPos(ax,xc[1,120]/1000,xc[1,240]/1000)

ax3.legend(loc ='upper right', bbox_to_anchor=(2,1))
ax8.set_xlabel('Alongshore distance (km)')
ax4.set_ylabel('Tracer mass per m ($10^6$ $[C]m^3$)')

plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1, figsize = (10,10),sharex=True)

tt = np.arange(19)


ax2.plot(tt/2.,np.sum(HCW-HCWNoC,axis=1)/1.E10,sns.xkcd_rgb["kelly green"],marker = 'o',label='Base $f=9.66E-5s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW17-HCW30,axis=1)/1.E10,sns.xkcd_rgb["forest"],marker = 'o',label='$\kappa_v=10^{-7}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW11-HCW24,axis=1)/1.E10,sns.xkcd_rgb["leaf"],marker = 'o',label='$\kappa_v=10^{-4}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW12-HCW25,axis=1)/1.E10,sns.xkcd_rgb["jade"],marker = 'o',label='$\kappa_v=10^{-3}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW043D-HCW30,axis=1)/1.E10,sns.xkcd_rgb["royal purple"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW053D-HCW30,axis=1)/1.E10,sns.xkcd_rgb["muted purple"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW063D-HCWNoC,axis=1)/1.E10,sns.xkcd_rgb["dark hot pink"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW073D-HCWNoC,axis=1)/1.E10,sns.xkcd_rgb["muted pink"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW18-HCW22,axis=1)/1.E10,sns.xkcd_rgb["sapphire"],marker = 'o',label='$f=7.68E-5s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW19-HCW23,axis=1)/1.E10,sns.xkcd_rgb["cerulean"],marker = 'o',label='f=4.84E-5s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW16-HCW28 ,axis=1)/1.E10,sns.xkcd_rgb["bright red"],marker = 'o',label='$N=3.0E-3s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW14-HCW26 ,axis=1)/1.E10,sns.xkcd_rgb["blood orange"],marker = 'o',label='$N=3.9E-3s^{-1}$')
ax2.plot(tt/2.,np.sum(HCW15-HCW27 ,axis=1)/1.E10,sns.xkcd_rgb["saffron"],marker = 'o',label='$N=6.3E-3s^{-1}$')

   
ax1.plot(tt/2.,np.sum(TrMass-TrMassNoC,axis=1)/1.E14,sns.xkcd_rgb["kelly green"],marker = 'o',label='Base $f=9.66E-5s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass17-TrMass30,axis=1)/1.E14,sns.xkcd_rgb["forest"],marker = 'o',label='$\kappa_v=10^{-7}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass11-TrMass24,axis=1)/1.E14,sns.xkcd_rgb["leaf"],marker = 'o',label='$\kappa_v=10^{-4}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass12-TrMass25,axis=1)/1.E14,sns.xkcd_rgb["jade"],marker = 'o',label='$\kappa_v=10^{-3}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass043D-TrMass30,axis=1)/1.E14,sns.xkcd_rgb["royal purple"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass053D-TrMass30,axis=1)/1.E14,sns.xkcd_rgb["muted purple"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass063D-TrMassNoC,axis=1)/1.E14,sns.xkcd_rgb["dark hot pink"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass073D-TrMassNoC,axis=1)/1.E14,sns.xkcd_rgb["muted pink"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass16-TrMass28 ,axis=1)/1.E14,sns.xkcd_rgb["bright red"],marker = 'o',label='$N=3.0E-3s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass14-TrMass26 ,axis=1)/1.E14,sns.xkcd_rgb["blood orange"],marker = 'o',label='$N=3.9E-3s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass15-TrMass27 ,axis=1)/1.E14,sns.xkcd_rgb["saffron"],marker = 'o',label='$N=6.3E-3s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass18-TrMass22,axis=1)/1.E14,sns.xkcd_rgb["sapphire"],marker = 'o',label='$f=7.68E-5s^{-1}$')
ax1.plot(tt/2.,np.sum(TrMass19-TrMass23,axis=1)/1.E14,sns.xkcd_rgb["cerulean"],marker = 'o',label='$f=4.84E-5s^{-1}$')
        
    
ax1.legend(loc ='upper right', bbox_to_anchor=(1.4,1))
ax2.set_xlabel('Day')
ax1.set_ylabel('Tracer mass per m ($10^{14}$ $[C]m^3$)')
ax2.set_ylabel('HCW volume per m ($10^{10}$ $m^3$)')
plt.tight_layout()
plt.show()

In [ ]:
sns.set_context('talk', font_scale=1.2)

fig,(ax1) = plt.subplots(1,1, figsize = (10,6),sharex=True)

    
    


tt = np.arange(19)
   
ax1.plot(tt/2.,(np.sum(TrMass,axis=1)/np.sum(TrMass, axis=1)[0])-
         (np.sum(TrMassNoC,axis=1)/np.sum(TrMassNoC,axis=1)[0]),
         sns.xkcd_rgb["kelly green"],marker = 'o',label='Base $f=9.66E-5s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass17,axis=1)/np.sum(TrMass17, axis=1)[0])-
         (np.sum(TrMass30,axis=1)/np.sum(TrMass30,axis=1)[0]),
         sns.xkcd_rgb["forest"],marker = 'o',label='$\kappa_v=10^{-7}m^2s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass11,axis=1)/np.sum(TrMass11, axis=1)[0])
         -(np.sum(TrMass24,axis=1)-np.sum(TrMass24,axis=1)[0]),
         sns.xkcd_rgb["leaf"],marker = 'o',label='$\kappa_v=10^{-4}m^2s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass12,axis=1)-np.sum(TrMass25,axis=1))/
         np.sum(TrMass12, axis=1)[0],sns.xkcd_rgb["jade"],marker = 'o',label='$\kappa_v=10^{-3}m^2s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass043D,axis=1)-np.sum(TrMass30,axis=1))/
         np.sum(TrMass043D, axis=1)[0],sns.xkcd_rgb["purplish"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-7}m^2s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass053D,axis=1)-np.sum(TrMass30,axis=1))/
         np.sum(TrMass053D, axis=1)[0],sns.xkcd_rgb["cerise"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-7}m^2s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass063D,axis=1)-np.sum(TrMassNoC,axis=1))/
         np.sum(TrMass063D, axis=1)[0],sns.xkcd_rgb["amethyst"],marker = 'o',label='$\kappa_vin=10^{-3},\kappa_vout=10^{-5}m^2s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass073D,axis=1)-np.sum(TrMassNoC,axis=1))/
         np.sum(TrMass073D, axis=1)[0],sns.xkcd_rgb["lavender pink"],marker = 'o',label='$\kappa_vin=10^{-4},\kappa_vout=10^{-5}m^2s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass16,axis=1)-np.sum(TrMass28 ,axis=1))/
         np.sum(TrMass16, axis=1)[0],sns.xkcd_rgb["nice blue"],marker = 'o',label='$N=3.0E-3s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass14,axis=1)-np.sum(TrMass26 ,axis=1))/
         np.sum(TrMass14, axis=1)[0],sns.xkcd_rgb["navy blue"],marker = 'o',label='$N=3.9E-3s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass15,axis=1)-np.sum(TrMass27 ,axis=1))/
         np.sum(TrMass15, axis=1)[0],sns.xkcd_rgb["sapphire"],marker = 'o',label='$N=6.3E-3s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass18,axis=1)-np.sum(TrMass22,axis=1))/
         np.sum(TrMass18, axis=1)[0],sns.xkcd_rgb["red orange"],marker = 'o',label='$f=7.68E-5s^{-1}$')

ax1.plot(tt/2.,(np.sum(TrMass19,axis=1)-np.sum(TrMass23,axis=1))/
         np.sum(TrMass19, axis=1)[0],sns.xkcd_rgb["tangerine"],marker = 'o',label='$f=4.84E-5s^{-1}$')
        
    
ax1.legend(loc ='upper right', bbox_to_anchor=(1.4,1))
ax1.set_xlabel('Day')
ax1.set_ylabel('$(M / M_0)-(M_{flat} / M_{0flat})$')
plt.tight_layout()
plt.show()

fig.savefig('/home/kramosmu/Documents/CMOS2016/Figures/TrMassOthers.pdf',format='pdf',dpi=1000,bbox_inches='tight')


### Process the rest of the runs and save dataframe


In [ ]:
# Process the rest of the runs to save pandas dataframe
Ptracers02 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run02/ptracersGlob.nc' 
Ptracers04 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run04/ptracersGlob.nc' 
Ptracers20 = '/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run20/ptracersGlob.nc' 

Tra = rout.getField(Ptracers02, 'Tr1')
Trb = rout.getField(Ptracers04, 'Tr1')
Trc = rout.getField(Ptracers20, 'Tr2')

HCW02,TrMass02 = HCW_Tracer_AlongShelf(Tra,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW04,TrMass04 = HCW_Tracer_AlongShelf(Trb,mask_NoC,30,rA,hFacC,drF,227,30,180,50)
HCW20,TrMass20 = HCW_Tracer_AlongShelf(Trc,mask_NoC,30,rA,hFacC,drF,227,30,180,50)


In [ ]:
runs = ['02','03','04','07','11','12','14','15','16','17','18','19','20','043D','053D','063D','073D']

DHCW_data = np.zeros(len(runs))
DTrMass_data = np.zeros(len(runs))
# HCW
DHCW_data[0] = ((np.sum(HCW02-HCW07,axis=1)[18]-np.sum(HCW02-HCW07,axis=1)[10])/(4*3600.0*24))
DHCW_data[1] = ((np.sum(HCW-HCW07,axis=1)[18]-np.sum(HCW-HCW07,axis=1)[10])/(4*3600.0*24))
DHCW_data[2] = ((np.sum(HCW04-HCW07,axis=1)[18]-np.sum(HCW04-HCW07,axis=1)[10])/(4*3600.0*24))
DHCW_data[3] = ((np.sum(HCWNoC-HCW07,axis=1)[18]-np.sum(HCWNoC-HCW07,axis=1)[10])/(4*3600.0*24))
DHCW_data[4] = ((np.sum(HCW11-HCW24,axis=1)[18]-np.sum(HCW11-HCW24,axis=1)[10])/(4*3600.0*24))
DHCW_data[5] = ((np.sum(HCW12-HCW25,axis=1)[18]-np.sum(HCW12-HCW25,axis=1)[10])/(4*3600.0*24))
DHCW_data[6] = ((np.sum(HCW14-HCW26,axis=1)[18]-np.sum(HCW14-HCW26,axis=1)[10])/(4*3600.0*24))
DHCW_data[7] = ((np.sum(HCW15-HCW27,axis=1)[18]-np.sum(HCW15-HCW27,axis=1)[10])/(4*3600.0*24))
DHCW_data[8] = ((np.sum(HCW16-HCW28,axis=1)[18]-np.sum(HCW16-HCW28,axis=1)[10])/(4*3600.0*24))
DHCW_data[9] = ((np.sum(HCW17-HCW30,axis=1)[18]-np.sum(HCW17-HCW30,axis=1)[10])/(4*3600.0*24))
DHCW_data[10] = ((np.sum(HCW18-HCW22,axis=1)[18]-np.sum(HCW18-HCW22,axis=1)[10])/(4*3600.0*24))
DHCW_data[11] = ((np.sum(HCW19-HCW23,axis=1)[18]-np.sum(HCW19-HCW23,axis=1)[10])/(4*3600.0*24))
DHCW_data[12] = ((np.sum(HCW20-HCW07,axis=1)[18]-np.sum(HCW20-HCW07,axis=1)[10])/(4*3600.0*24))

DHCW_data[13] = ((np.sum(HCW043D-HCW30,axis=1)[18]-np.sum(HCW043D-HCW30,axis=1)[10])/(4*3600.0*24))
DHCW_data[14] = ((np.sum(HCW053D-HCW30,axis=1)[18]-np.sum(HCW053D-HCW30,axis=1)[10])/(4*3600.0*24))
DHCW_data[15] = ((np.sum(HCW063D-HCW07,axis=1)[18]-np.sum(HCW063D-HCW07,axis=1)[10])/(4*3600.0*24))
DHCW_data[16] = ((np.sum(HCW073D-HCW07,axis=1)[18]-np.sum(HCW073D-HCW07,axis=1)[10])/(4*3600.0*24))

 

#Tracer mass
DTrMass_data[0] = ((np.sum(TrMass02-TrMass07,axis=1)[18]-np.sum(TrMass02-TrMass07,axis=1)[10])/(4*3600.0*24))
DTrMass_data[1] =((np.sum(TrMass-TrMass07,axis=1)[18]-np.sum(TrMass-TrMass07,axis=1)[10])/(4*3600.0*24))
DTrMass_data[2] =((np.sum(TrMass04-TrMass07,axis=1)[18]-np.sum(TrMass04-TrMass07,axis=1)[10])/(4*3600.0*24))
DTrMass_data[3] =((np.sum(TrMassNoC-TrMass07,axis=1)[18]-np.sum(TrMassNoC-TrMass07,axis=1)[10])/(4*3600.0*24))
DTrMass_data[4] =((np.sum(TrMass11-TrMass24,axis=1)[18]-np.sum(TrMass11-TrMass24,axis=1)[10])/(4*3600.0*24))
DTrMass_data[5] =((np.sum(TrMass12-TrMass25,axis=1)[18]-np.sum(TrMass12-TrMass25,axis=1)[10])/(4*3600.0*24))
DTrMass_data[6] =((np.sum(TrMass14-TrMass26,axis=1)[18]-np.sum(TrMass14-TrMass26,axis=1)[10])/(4*3600.0*24))
DTrMass_data[7] =((np.sum(TrMass15-TrMass27,axis=1)[18]-np.sum(TrMass15-TrMass27,axis=1)[10])/(4*3600.0*24))
DTrMass_data[8] =((np.sum(TrMass16-TrMass28,axis=1)[18]-np.sum(TrMass16-TrMass28,axis=1)[10])/(4*3600.0*24))
DTrMass_data[9] =((np.sum(TrMass17-TrMass30,axis=1)[18]-np.sum(TrMass17-TrMass30,axis=1)[10])/(4*3600.0*24))
DTrMass_data[10] =((np.sum(TrMass18-TrMass22,axis=1)[18]-np.sum(TrMass18-TrMass22,axis=1)[10])/(4*3600.0*24))
DTrMass_data[11] =((np.sum(TrMass19-TrMass23,axis=1)[18]-np.sum(TrMass19-TrMass23,axis=1)[10])/(4*3600.0*24))
DTrMass_data[12] =((np.sum(TrMass20-TrMass07,axis=1)[18]-np.sum(TrMass20-TrMass07,axis=1)[10])/(4*3600.0*24))

 
DTrMass_data[13] =((np.sum(TrMass043D-TrMass30,axis=1)[18]-np.sum(TrMass043D-TrMass30,axis=1)[10])/(4*3600.0*24))
DTrMass_data[14] =((np.sum(TrMass053D-TrMass30,axis=1)[18]-np.sum(TrMass053D-TrMass30,axis=1)[10])/(4*3600.0*24))
DTrMass_data[15] =((np.sum(TrMass063D-TrMass07,axis=1)[18]-np.sum(TrMass063D-TrMass07,axis=1)[10])/(4*3600.0*24))
DTrMass_data[16] =((np.sum(TrMass073D-TrMass07,axis=1)[18]-np.sum(TrMass073D-TrMass07,axis=1)[10])/(4*3600.0*24))

In [ ]:
# save data frame
raw_data = {'RunNum':runs,'deltaHCW': DHCW_data,'deltaTrMass': DTrMass_data}

df = pd.DataFrame(raw_data, columns = ['eRunNum','deltaHCW','deltaTrMass'])

filename = ('results/metricsDataFrames/DeltaTrOnShelfCanyonEffect.csv' )
df.to_csv(filename)